# **Automated Chatbot for Data Query and Visualization**


- By **Prajna Mondal**

Project Summary -

An automated chatbot for data queries and visualization is an AI-powered system that can understand user questions related to data, fetch relevant information from connected data sources, and generate visual charts or graphs on demand. Using tools like Python, GPT, chatbot frameworks, and visualization libraries, the chatbot can interpret natural language queries, run the necessary data analysis, and interactively present results. This provides easy, user-friendly access to business insights without requiring technical skills. Such chatbots are widely used in customer service, business analytics, reporting, and decision-making.



# IMPORT LIBRARIES

In [16]:
#Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# DATASET LOADING

In [21]:
# Load Dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
# Read the excel file
import pandas as pd
df = pd.read_excel("/content/drive/MyDrive/Product-Sales-Region.xlsx")
df.head()

,Date,Region,Product,Quantity,UnitPrice,StoreLocation,CustomerType,Discount,Salesperson,TotalPrice,PaymentMethod,Promotion,Returned,OrderID,CustomerName,ShippingCost,OrderDate,DeliveryDate,RegionManager
0,2023-02-23,East,Laptop,14,163.60,Store B,Wholesale,0.00,Eva,2290.400,Online,FREESHIP,0,REG100000,Cust 6583,43.34,2023-02-23,2023-02-27,Eric
1,2024-12-19,South,Phone,1,544.01,Store A,Retail,0.00,Alice,544.010,Gift Card,SAVE10,0,REG100001,Cust 2144,5.30,2024-12-19,2024-12-28,Sophie
2,2023-05-10,North,Desk,14,346.18,Store B,Wholesale,0.10,Alice,4361.868,Online,WINTER15,0,REG100002,Cust 5998,20.46,2023-05-10,2023-05-19,Ryan
3,2025-02-26,Central,Chair,18,384.82,Store A,Wholesale,0.15,Frank,5887.746,Gift Card,FREESHIP,0,REG100003,Cust 7136,27.95,2025-02-26,2025-03-02,Cameron
4,2023-06-24,East,Desk,18,237.76,Store C,Retail,0.00,Carlos,4279.680,Online,SAVE10,0,REG100004,Cust 6506,5.73,2023-06-24,2023-06-27,Eric


# DATA CLEANING AND PREPROCESSING

In [ ]:
#Basic Data Info
print("🔹 Dataset Shape:")
print(df.shape)

print("🔹 Dataset Info:")
df.info()


🔹 Dataset Shape:
(1500, 19)
🔹 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           1500 non-null   datetime64[ns]
 1   Region         1500 non-null   object        
 2   Product        1500 non-null   object        
 3   Quantity       1500 non-null   int64         
 4   UnitPrice      1500 non-null   float64       
 5   StoreLocation  1500 non-null   object        
 6   CustomerType   1500 non-null   object        
 7   Discount       1500 non-null   float64       
 8   Salesperson    1500 non-null   object        
 9   TotalPrice     1500 non-null   float64       
 10  PaymentMethod  1500 non-null   object        
 11  Promotion      1130 non-null   object        
 12  Returned       1500 non-null   int64         
 13  OrderID        1500 non-null   object        
 14  CustomerName   1500 non-null

In [ ]:
print("🔹 Summary Stats:")
display(df.describe())

🔹 Summary Stats:


,Date,Quantity,UnitPrice,Discount,TotalPrice,Returned,ShippingCost,OrderDate,DeliveryDate
count,1500,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500,1500
mean,2024-04-07 21:35:02.400000256,10.410667,298.826947,0.073133,2919.994952,0.248000,27.507293,2024-04-07 21:35:02.400000256,2024-04-13 22:36:28.799999744
min,2023-01-01 00:00:00,1.000000,5.520000,0.000000,6.970000,0.000000,5.010000,2023-01-01 00:00:00,2023-01-08 00:00:00
25%,2023-08-20 00:00:00,5.000000,151.020000,0.000000,867.136250,0.000000,16.700000,2023-08-20 00:00:00,2023-08-25 12:00:00
50%,2024-04-08 12:00:00,11.000000,294.740000,0.050000,2174.724000,0.000000,27.100000,2024-04-08 12:00:00,2024-04-15 12:00:00
75%,2024-12-07 06:00:00,15.000000,446.702500,0.100000,4414.723500,0.000000,38.635000,2024-12-07 06:00:00,2024-12-13 06:00:00
max,2025-06-30 00:00:00,20.000000,599.720000,0.150000,11077.000000,1.000000,49.980000,2025-06-30 00:00:00,2025-07-10 00:00:00
std,NaN,5.735732,169.100075,0.055979,2522.789977,0.431996,13.093453,NaN,NaN


In [ ]:
# Check missing values
print("🔹 Missing Values:")
print(df.isnull().sum())

#Drop rows with missing values
df_dropna = df.dropna()

#Fill missing values with mean/median/mode
df_fill = df.copy()

for column in df_fill.columns:
    if df_fill[column].dtype != 'object':
        df_fill[column].fillna(df_fill[column].mean(), inplace=True)
    else:
        df_fill[column].fillna(df_fill[column].mode()[0], inplace=True)

print("🔹 Missing Values After Cleaning:")
print(df_fill.isnull().sum())

🔹 Missing Values:
Date               0
Region             0
Product            0
Quantity           0
UnitPrice          0
StoreLocation      0
CustomerType       0
Discount           0
Salesperson        0
TotalPrice         0
PaymentMethod      0
Promotion        370
Returned           0
OrderID            0
CustomerName       0
ShippingCost       0
OrderDate          0
DeliveryDate       0
RegionManager      0
month              0
month_name         0
dtype: int64
🔹 Missing Values After Cleaning:
Date             0
Region           0
Product          0
Quantity         0
UnitPrice        0
StoreLocation    0
CustomerType     0
Discount         0
Salesperson      0
TotalPrice       0
PaymentMethod    0
Promotion        0
Returned         0
OrderID          0
CustomerName     0
ShippingCost     0
OrderDate        0
DeliveryDate     0
RegionManager    0
month            0
month_name       0
dtype: int64


In [ ]:
#Removing Duplicates
df_clean = df_fill.drop_duplicates()

print("🔹 Removed Duplicates:")
print(df_clean.duplicated().sum())

🔹 Removed Duplicates:
0


In [ ]:
#Handling Outliers
65\ b
df_numeric = df_clean.select_dtypes(include=np.number)

Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1

# Filter out the outliers based on the numerical columns
df_no_outliers = df_clean[~((df_numeric < (Q1 - 1.5 * IQR)) | (df_numeric > (Q3 + 1.5 * IQR))).any(axis=1)]

print("🔹 Outliers Removed Shape:", df_no_outliers.shape)

🔹 Outliers Removed Shape: (1114, 19)


In [ ]:
#Feature Engineering
import pandas as pd
import numpy as np

# Ensure df_fill is defined (copied from DIFM3UpZCuSg)
df_fill = df.copy()
for column in df_fill.columns:
    if df_fill[column].dtype != 'object':
        df_fill[column] = df_fill[column].fillna(df_fill[column].mean())
    else:
        df_fill[column] = df_fill[column].fillna(df_fill[column].mode()[0])

# Ensure df_clean is defined (copied from 7Bd452H6Dvt4)
df_clean = df_fill.drop_duplicates().copy()

# Handling Outliers (original content from xMaMCbacD6TG)
df_numeric = df_clean.select_dtypes(include=np.number)
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1
# Explicitly create a copy to avoid SettingWithCopyWarning
df_no_outliers = df_clean[~((df_numeric < (Q1 - 1.5 * IQR)) | (df_numeric > (Q3 + 1.5 * IQR))).any(axis=1)].copy()

# Extract month and store it in a new column for the cleaned dataset
df_no_outliers['month'] = df_no_outliers['Date'].dt.month
df_no_outliers['month_name'] = df_no_outliers['Date'].dt.strftime('%B')

# Show result
display(df_no_outliers.head())

,Date,Region,Product,Quantity,UnitPrice,StoreLocation,CustomerType,Discount,Salesperson,TotalPrice,...,Promotion,Returned,OrderID,CustomerName,ShippingCost,OrderDate,DeliveryDate,RegionManager,month,month_name
0,2023-02-23,East,Laptop,14,163.60,Store B,Wholesale,0.00,Eva,2290.400,...,FREESHIP,0,REG100000,Cust 6583,43.34,2023-02-23,2023-02-27,Eric,2,February
1,2024-12-19,South,Phone,1,544.01,Store A,Retail,0.00,Alice,544.010,...,SAVE10,0,REG100001,Cust 2144,5.30,2024-12-19,2024-12-28,Sophie,12,December
2,2023-05-10,North,Desk,14,346.18,Store B,Wholesale,0.10,Alice,4361.868,...,WINTER15,0,REG100002,Cust 5998,20.46,2023-05-10,2023-05-19,Ryan,5,May
3,2025-02-26,Central,Chair,18,384.82,Store A,Wholesale,0.15,Frank,5887.746,...,FREESHIP,0,REG100003,Cust 7136,27.95,2025-02-26,2025-03-02,Cameron,2,February
4,2023-06-24,East,Desk,18,237.76,Store C,Retail,0.00,Carlos,4279.680,...,SAVE10,0,REG100004,Cust 6506,5.73,2023-06-24,2023-06-27,Eric,6,June


In [ ]:
# STEP 8: Final Cleaned Dataset
from google.colab import files

print("🔹 Final Cleaned Dataset:")
display(df_no_outliers.head())

# Save cleaned file
df_no_outliers.to_csv("cleaned_dataset.csv", index=False)

print("✅ Cleaning Completed! Download your cleaned file below ⬇️")
files.download("cleaned_dataset.csv")

🔹 Final Cleaned Dataset:


,Date,Region,Product,Quantity,UnitPrice,StoreLocation,CustomerType,Discount,Salesperson,TotalPrice,...,Promotion,Returned,OrderID,CustomerName,ShippingCost,OrderDate,DeliveryDate,RegionManager,month,month_name
0,2023-02-23,East,Laptop,14,163.60,Store B,Wholesale,0.00,Eva,2290.400,...,FREESHIP,0,REG100000,Cust 6583,43.34,2023-02-23,2023-02-27,Eric,2,February
1,2024-12-19,South,Phone,1,544.01,Store A,Retail,0.00,Alice,544.010,...,SAVE10,0,REG100001,Cust 2144,5.30,2024-12-19,2024-12-28,Sophie,12,December
2,2023-05-10,North,Desk,14,346.18,Store B,Wholesale,0.10,Alice,4361.868,...,WINTER15,0,REG100002,Cust 5998,20.46,2023-05-10,2023-05-19,Ryan,5,May
3,2025-02-26,Central,Chair,18,384.82,Store A,Wholesale,0.15,Frank,5887.746,...,FREESHIP,0,REG100003,Cust 7136,27.95,2025-02-26,2025-03-02,Cameron,2,February
4,2023-06-24,East,Desk,18,237.76,Store C,Retail,0.00,Carlos,4279.680,...,SAVE10,0,REG100004,Cust 6506,5.73,2023-06-24,2023-06-27,Eric,6,June


✅ Cleaning Completed! Download your cleaned file below ⬇️


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **END OF PAPER **